In [83]:
using JuMP, Gurobi, Test, Combinatorics, LightGraphs, TimerOutputs

# function solveMP()
const gurobi_env = Gurobi.Env()
# setparams!(gurobi_env, OutputFlag = 0)

ins = 2 #ARGS[1] #:10
dataSet = "N20_d40_"
println("Running...")
global prob = "" #"L"
#     include("./TestInstances/"*fileName*".jl")
global q = []
include("./TestInstances/"*dataSet*string(ins)*".jl")
global numArcs = length(arcs[:,1])
global lb = 1
global ub = 999
# println("1")
function setQ(prob)
    global lb, ub, numArcs, q
    if prob == "L"
        lb = 1
        ub = 333
    elseif prob == "M"
        lb = 334
        ub = 666
    elseif prob == "H"
        lb = 667
        ub = 999
    end
    new_q = rand(lb:ub, numArcs)./1000
       
    return new_q
end
# println("2")
if prob != ""
    q = setQ(prob)
end
println("q = ", q)
# q = [0.28, 0.23, 0.033, 0.257, 0.208, 0.037, 0.009, 0.269, 0.302, 0.167, 0.098, 0.288, 0.187]
# println("3")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY

include("./functionEnumFeasX.jl")
include("./functionFindQ.jl")
include("./functionGetCost_SP.jl")

include("./functionShortestPathBellmanFord.jl")
include("./functionGenMonitoring.jl")
include("./functionLP_findM.jl")

#     global X_feas = EnumX(arcs, b_x, numArcs, d_x)
#     global EnumY = EnumX(arcs, b_y, numArcs, d_y)
M_start = findall(d_x.<=b_x)
global M_set = [[M_start[1]]]#[EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
global numY = 1 
global R = -sum(log10(1-q_) for q_ in q)  #10^6#0^6
println("R = ", R)
path, gx = shortestPath_BellmanFord(q, arcs, numArcs, origin, destination)
arcsinPath = findall(path.>0)
global P_set = [arcsinPath]

global y_sol = []
#     global 
global numPaths = 1
global newP = true
global newM = true
global iter = 0
global theta_inc = 1
global x_inc = []
global y_inc = []
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
conRefNum = 20# max(200,2*length(P_set))
con = Array{JuMP.ConstraintRef}(undef, conRefNum)
global k = 0
global rootOptGap = 0
model = direct_model(Gurobi.Optimizer())
set_optimizer_attribute(model, "LazyConstraints", 1)
# set_optimizer_attribute(model, "OutputFlag", 0)
global loc = "./Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/"
# set_optimizer_attribute(model, "LogFile", loc*"L_"*dataSet*prob*"_"*string(ins)*".log")

@variable(model, x[1:numArcs], Bin)
@variable(model, theta >= 1-R)#1e6)
@objective(model, Min, theta)
@constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    
global last_theta = -1e6 - 1
global start = time()
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y, y_sol
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
    global theta_inc, x_inc, y_inc
    global k, last_theta
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        iter += 1
        MIPGAP = 0
        if cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cdouble}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_NODCNT, resultP)
            node = resultP[]
#             println("\n\tMIPNODE_NODCNT ", node)
            if node == 0
                resultP = Ref{Cdouble}()
                GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_OBJBST, resultP)
                obj = resultP[]
#                 println("\tMIPNODE_OBJBST ", obj)
                resultP = Ref{Cdouble}()
                GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_OBJBND, resultP)
                objBound = resultP[]
#                 println("\tMIPNODE_OBJBND ", objBound)
                MIPGAP = (obj - objBound)/obj*100
#                 println("\tMIPGAP = ", MIPGAP)
                if resultP[] != GRB_OPTIMAL
                    return  # Solution is something other than optimal.
                end
            end
        end
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPSOL #|| cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            # Before querying `callback_value`, you must call:
            ret = GRBcbget(cb_data, cb_where, GRB_CB_MIPSOL_OBJBST, resultP)
            Gurobi.load_callback_variable_primal(cb_data, cb_where)
#         if resultP[] != GRB_OPTIMAL
#             return  # Solution is something other than optimal.
#         end
#         if resultP == GRB_OPTIMAL
#             println("\niter = ", iter)
            x_val = callback_value.(Ref(cb_data), x)
            theta_val = callback_value(cb_data, theta)
#             println("x_val = ", findall(x_val.>0))#, "\t theta_val", theta_val)
            P_set, M_set, lambda, pi_, inner_Obj, y_sol = solveLP_findM(x_val)
#             println("M = ", length(M_set), "\t P = ", length(P_set)) 
#             println("P = ", P_set)
#             println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
#             println("lambda = ", lambda, "\t pi_ = ", pi_)
            
            numPaths = length(P_set)
            numY = length(M_set)
            lambda_pos = findall(lambda.>0)
            
            TOL = 10^(-3)
#             println("(inner_Obj - theta_val) = ", (inner_Obj - theta_val))
            
#             println("TOL = ", TOL)
            if (inner_Obj - theta_val) > TOL  
                k+=1 
                con_ = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println("Added con")
#                 println("Type = ", typeof(con_))
#             cut_vio = (sum(lambda[i]*(-R)*sum(x_val[a] for a in P_set[i]) for i=1:numPaths) + pi_) - theta_val
#                 println("Add con : cut violation = ", cut_vio)
#                 println(theta, ">=", sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                
                w_con = MOI.submit(model, MOI.LazyConstraint(cb_data), con_)

#                 X_set = []
#                 for i in lambda_pos
#                     X_set = vcat(X_set,P_set[i])
#                 end
#                 unique!(X_set)
#                 c2 = @build_constraint(sum(x[a] for a in X_set) >= 1)
#                 MOI.submit(model, MOI.LazyConstraint(cb_data), c2)
            end
#             if inner_Obj < theta_inc
# #                 println(iter, "Updating Incumbent Sol : ", inner_Obj, " \t ", findall(x_val.>0))
#                 theta_inc = inner_Obj
#                 x_inc = x_val
#                 y_inc = y_sol
#             end
            
#         if last_theta > theta_val
#             GRBterminate(backend(model))
#         else
#             last_theta = theta_val
#         end
#         end 
    end
        
    return
end
#     You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
#     println(model)
optimize!(model)
println("theta_inc ", theta_inc)
println("x_inc ", x_inc)
#     println(model)
totalTime = time() - start
x_sol = JuMP.value.(x)
theta_sol = JuMP.value.(theta)
println("dataSet ", dataSet*prob)
println("x = ", x_sol)
println("x_i = ", findall(x_sol.>0))
println("theta = ", theta_sol)
println("M_set = ", length(M_set))
y_pos = findall(y_sol .> 0)
println("y = ", y_sol[y_pos])
println("M_pos = ", M_set[y_pos])
nodeCount = MOI.get(model, MOI.NodeCount())
println("Node count = ", nodeCount)
println("Total time = ", totalTime)
# timesFile = open(loc*"O_"*dataSet*prob*".txt", "a")
# println(timesFile, dataSet*prob, "; Ins ", ins, "; Time ", totalTime, "; theta ", theta_sol,"; x_sol ", findall(x_sol.>0),"; M_select ", M_set[y_pos], "; y_sol ", y_sol[y_pos], "; Iter ", iter, "; M_len ", length(M_set), "; P_len ", length(P_set), "; NodeCount ", nodeCount)
# close(timesFile)

# if prob != ""
#     qFile = open(loc*"q_"*dataSet*prob*".txt", "a")
#     println(qFile, dataSet*prob, "; Ins ", ins, "; q ", q)
#     close(qFile)
# end

Academic license - for non-commercial use only - expires 2022-12-04
Running...
q = [0.364, 0.6, 0.525, 0.358, 0.642, 0.915, 0.786, 0.639, 0.797, 0.203, 0.013, 0.891, 0.985, 0.588, 0.076, 0.493, 0.142, 0.337, 0.382, 0.222, 0.221, 0.741, 0.422, 0.269, 0.18, 0.271, 0.755, 0.031, 0.125, 0.844, 0.324, 0.093, 0.091, 0.704, 0.142, 0.181, 0.872, 0.564, 0.375, 0.191, 0.267, 0.393, 0.722, 0.182, 0.029, 0.476, 0.634, 0.155, 0.736, 0.377, 0.485, 0.681, 0.511, 0.484, 0.584, 0.734, 0.411, 0.953, 0.588, 0.685, 0.762, 0.794, 0.636, 0.498, 0.795, 0.104, 0.947, 0.487, 0.267, 0.828, 0.431, 0.42, 0.468, 0.078, 0.619, 0.863, 0.158, 0.836, 0.012, 0.712, 0.231, 0.12, 0.964, 0.659, 0.67, 0.068, 0.831, 0.584, 0.041, 0.921, 0.666, 0.811, 0.187, 0.948, 0.701, 0.516, 0.402, 0.834, 0.438, 0.141, 0.813, 0.871, 0.713, 0.09, 0.015, 0.049, 0.815, 0.417, 0.488, 0.55, 0.391, 0.645, 0.319, 0.206, 0.697, 0.242, 0.327, 0.135, 0.539, 0.25, 0.392, 0.975, 0.006, 0.413, 0.888, 0.53, 0.992, 0.388, 0.19, 0.3, 0.27, 0.927, 0.95, 

0.562, 0.956, 0.086, 0.796, 0.452, 0.754, 0.51, 0.168, 0.719, 0.752, 0.036, 0.865, 0.697, 0.739, 0.998, 0.483, 0.076, 0.447, 0.858, 0.085, 0.452]
R = 65.6274806652216
Academic license - for non-commercial use only - expires 2022-12-04
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 156 columns and 155 nonzeros
Model fingerprint: 0x913f7144
Variable types: 1 continuous, 155 integer (155 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [6e+01, 6e+01]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 1 rows, 156 columns, 155 nonzeros
Variable types: 1 continuous, 155 integer (155 binary)

Root relaxation: objective -6.462748e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [79]:
 0.8786266988960166

0.8786266988960166

In [82]:
findall(x_inc.>0)

1-element Array{Int64,1}:
 3